# Selección de atributos 

In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]

Para garantizar la reproducibilidad (esto es, que al ejecutar este notebook otra vez los resultados sean idénticos) vamos a fijar la semilla que usa scikit learn para hacer particiones aleatorias. Para esto tenemos que fijar la semilla (seed) cada vez que llamemos a la aplicacion.

In [3]:
 np.random.seed(42)

# Selección de variables

En esta sección vamos a ver las distintas técnicas de selección de variables, que son las técnicas que nos permiten filtrar las variables que tenemos y elegir aquellas que realmente están ayudando al modelo.

¿Por qué querriamos eliminar variables? Generalmente, cuanta más información tenemos sobre cada observación mejor será la calidad de nuestras predicciones (cuanto más sepamos, más facil será diferenciar unos casos de otros). No obstante, esto no tiene por que ser así en todas las situaciones, hay veces que ciertas variables pueden ser contraproducentes y hacer que nuestros modelos funcionen peor, por ejemplo debido a errores de medición.

Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el número de variables independientes reduce la complejidad del modelo, reduciendo así el efecto de sobreajuste del modelo. Además, tener menos variables independientes y modelos más simples significa que nuestros modelos son más rápidos de entrenar, y podemos entrenarlos con más datos en las mismas máquinas.

## Cargamos los datos

Para esta sección vamos a usar el [Ames Housing dataset](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) que es una versión más moderna y con muchas más variables que el Boston Housing Dataset. Tiene información sobre casas en la ciudad de Ames, en Iowa, y su precio de venta. Este [link](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) contiene la descripción de las variables en inglés.

In [4]:
ames = pd.read_csv("data/ames.csv").drop("id_parcela", axis=1) # no necesitamos el identificador de la parcela

In [5]:
ames.shape

(2930, 80)

In [6]:
ames.head()

,tipo_construccion,tipo_zona,perimetro_conectado_calle,area_parcela,tipo_acceso,tipo_acceso_trasero,forma_parcela,LandContour,tipo_instalaciones,config_parcela,...,area_piscina,calidad_piscina,calidad_valla,atributo_miscelaneo,valor_atributo_miscelaneo,mes_venta,ano_venta,tipo_venta,condicion_venta,precio_venta
0,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [7]:
ames.shape

(2930, 80)

In [8]:
ames.sample(3).T

,1357,2367,2822
tipo_construccion,70,160,60
tipo_zona,RM,RM,RL
perimetro_conectado_calle,NaN,21,62
area_parcela,5100,1890,7162
tipo_acceso,Pave,Pave,Pave
...,...,...,...
mes_venta,6,7,5
ano_venta,2008,2006,2006
tipo_venta,WD,WD,WD
condicion_venta,Normal,Normal,Normal


In [9]:
ames.precio_venta.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: precio_venta, dtype: float64

In [10]:
ames.dtypes

tipo_construccion              int64
tipo_zona                     object
perimetro_conectado_calle    float64
area_parcela                   int64
tipo_acceso                   object
                              ...   
mes_venta                      int64
ano_venta                      int64
tipo_venta                    object
condicion_venta               object
precio_venta                   int64
Length: 80, dtype: object

## Procesado de datos

Para poder utilizar este dataset tenemos que transformar las variables categóricas en numéricas. En vez de crear nuestro propio `ColumnSelector` vamos a usar el de la libreria auxiliar `mlxtend`

In [11]:
variables_independientes = ames.drop("precio_venta", axis=1).columns
variable_objetivo = ["precio_venta"]

In [12]:
datos_numericos = ames[variables_independientes].select_dtypes([int, float])
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns

col_numericas = datos_numericos.columns

# para crear este diccionario he ido a la descripcion del dataset
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}
col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]


In [13]:
datos_numericos.head()

,perimetro_conectado_calle,area_revestimiento,area_sotano_habitable1,area_sotano_habitable2,area_sotano_inhabitable,area_sotano_total,n_aseos_sotanos,n_medios_aseos_sotanos,ano_construccion_garaje,n_coches_garaje,area_garage
0,141.0,112.0,639.0,0.0,441.0,1080.0,1.0,0.0,1960.0,2.0,528.0
1,80.0,0.0,468.0,144.0,270.0,882.0,0.0,0.0,1961.0,1.0,730.0
2,81.0,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1958.0,1.0,312.0
3,93.0,0.0,1065.0,0.0,1045.0,2110.0,1.0,0.0,1968.0,2.0,522.0
4,74.0,0.0,791.0,0.0,137.0,928.0,0.0,0.0,1997.0,2.0,482.0


In [14]:
datos_categoricos.head()

,condicion_venta,LandContour,tipo_calefaccion,material_tejado,tipo_garaje,tipo_venta,sistema_electrico,barrio,tipo_casa,tipo_acceso_trasero,...,atributo_miscelaneo,cercania_carretera2,cercania_carretera1,tipo_acceso,config_parcela,aire_acondicionado_central,tipo_edificio,material_exterior2,tipo_revestimiento,material_exterior1
0,Normal,Lvl,GasA,CompShg,Attchd,WD,SBrkr,NAmes,1Story,NaN,...,NaN,Norm,Norm,Pave,Corner,Y,1Fam,Plywood,Stone,BrkFace
1,Normal,Lvl,GasA,CompShg,Attchd,WD,SBrkr,NAmes,1Story,NaN,...,NaN,Norm,Feedr,Pave,Inside,Y,1Fam,VinylSd,None,VinylSd
2,Normal,Lvl,GasA,CompShg,Attchd,WD,SBrkr,NAmes,1Story,NaN,...,Gar2,Norm,Norm,Pave,Corner,Y,1Fam,Wd Sdng,BrkFace,Wd Sdng
3,Normal,Lvl,GasA,CompShg,Attchd,WD,SBrkr,NAmes,1Story,NaN,...,NaN,Norm,Norm,Pave,Corner,Y,1Fam,BrkFace,None,BrkFace
4,Normal,Lvl,GasA,CompShg,Attchd,WD,SBrkr,Gilbert,2Story,NaN,...,NaN,Norm,Norm,Pave,Inside,Y,1Fam,VinylSd,None,VinylSd


In [15]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,TA,Typ,Gd,Fin,TA,TA,P,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,Gd,Gd,BLQ,Unf,Fa
1,TA,Typ,NaN,Unf,TA,TA,Y,NaN,MnPrv,Reg,AllPub,Gtl,TA,TA,TA,TA,No,Rec,LwQ,TA
2,Gd,Typ,NaN,Unf,TA,TA,Y,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,TA,No,ALQ,Unf,TA
3,Ex,Typ,TA,Fin,TA,TA,Y,NaN,NaN,Reg,AllPub,Gtl,Gd,TA,TA,TA,No,ALQ,Unf,Ex
4,TA,Typ,TA,Fin,TA,TA,Y,NaN,MnPrv,IR1,AllPub,Gtl,TA,TA,Gd,TA,No,GLQ,Unf,Gd


Nos aseguramos que todas las columnas están clasificadas como categóricas o numéricas

In [16]:
[col for col in ames.columns if col not in datos_numericos.columns and\
 col not in datos_categoricos.columns and \
 col not in datos_ordinales.columns]

['tipo_construccion',
 'area_parcela',
 'calidad_general',
 'condicion_general',
 'fecha_construccion',
 'ano_remodelado',
 'area_piso1',
 'area_piso2',
 'area_suelos_baja_calidad',
 'area_habitable_sobre_suelo',
 'n_aseos_sobre_suelo',
 'n_medios_aseos_sobre_suelo',
 'n_dormitorios_sobre_suelo',
 'n_cocinas',
 'n_habitaciones_sobre_suelo',
 'n_chimeneas',
 'area_terraza_madera',
 'area_porche_abierto',
 'area_porche_cerrado',
 'area_porche_3estaciones',
 'area_porche_tapado',
 'area_piscina',
 'valor_atributo_miscelaneo',
 'mes_venta',
 'ano_venta',
 'precio_venta']

## Variables numéricas

con las variables numéricas muy sencillo, vamos a eliminar valores inexistentes (reemplazandolos pr normalizarlas

In [23]:
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer


# Recordatorio: los transformadores de sklearn devuelven numpy arrays, no dataframes
datos_numericos_imputados_normalizados = pd.DataFrame(
    normalize(SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(datos_numericos)),
    columns=datos_numericos.columns
)

### Variables categóricas

Con las variables categóricas tenemos dos opciones principalmente:
- Usar `sklearn.preprocessing.LabelBinarizer` para hacer un 1 hot encoding y codificarlas como vectores
- Usar la función get_dummies de pandas.

En general la opción recomendada es la de usar `LabelBinarizer` ya que esto nos crea un transformador de scikit learn que podemos usar en pipelines y para transformar nuevas observaciones. Sin embargo dado que esta sección es sobre selección de variables y para no hacerla demasiado larga, directamente voy a usar `pd.get_dummies`.

In [24]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)

In [25]:
datos_categoricos_dummy.shape

(2930, 153)

### Variables ordinales

Yendo al [diccionario de datos](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) (donde se describen todas las variables) hay varias variables que son ordinales (Ordinal), básicamente evaluaciones de una condición de la casa definidas en una escala no numérica pero que tiene cierto orden (de "mejor" a "peor").

Scikit-learn no tiene un Codificador ordinal (hasta ahora hemos usado el label encoder.) Tenemos tres opciones:

1. Implementar nuestro propio `OrdinalEncoder`.
2. Utilizar la implementación de `OrdinalEncoder` del paquete [scikit-learn.contrib.categorical_encoding](https://github.com/scikit-learn-contrib/categorical-encoding), que podemos instalar con 

```conda install -c conda-forge category_encoders
ó
pip install category_encoders
```
3. Ya que estamos usando pandas, podemos convertir las variables ordinales a categorías de pandas, especificar sus niveles y convertirlas a su valor numérico. Vamos a usar esta opción.

In [26]:
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}

In [27]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)

<ipython-input-27-6cb75cffb96c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_ordinales[columna_ordinal] = (


In [28]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,2,6,4,3,3,3,1,-1,-1,2,3,2,2,2,3,4,4,4,1,1
1,2,6,-1,1,3,3,2,-1,3,3,3,2,2,2,3,3,1,3,2,2
2,3,6,-1,1,3,3,2,-1,-1,2,3,2,2,2,3,3,1,5,1,2
3,4,6,3,3,3,3,2,-1,-1,3,3,2,3,2,3,3,1,5,1,4
4,2,6,3,3,3,3,2,-1,3,2,3,2,2,2,4,3,1,6,1,3


In [29]:
### Unimos los tres tipos de datos

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)

In [30]:
ames_procesado.shape

(2930, 184)

Ya tenemos un dataset preparado para poder entrenar modelos. Este dataset tiene una complejidad dimensional alta ( 209 variables independientes), por lo que vamos a usar técnicas de selección de variables para reducirla.

Antes que nada vamos a ver que errores obtenemos con diversos modelos entrenando con el dataset con todas las variables.

In [31]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


ames_X = ames_procesado
ames_y = ames[variable_objetivo]

Vamos a usar la función `cross_validate` que es una versión más flexible que `cross_val_score`. Evaluaremos usando la raíz del error cuadrático medio (RMSE)

In [32]:
def rmse(y_real, y_pred):
    return np.sqrt(mean_squared_error(y_real, y_pred))

def rmse_cv(estimator, X, y):
    y_pred = estimator.predict(X)
    return rmse(y, y_pred)

res = cross_validate(LinearRegression(), ames_procesado, ames[variable_objetivo],
                     scoring=rmse_cv, n_jobs=-1, cv=10)

Dicha función devuelve no solo las puntuaciones en los datos de testing, sino también en los datos de entrenamiento. Además, nos devuelve el tiempo que tarda dicho estimador en ajustar el modelo y en hacer predicciones con el mismo.

In [33]:
res

{'fit_time': array([0.04389262, 0.05286813, 0.04289556, 0.04363036, 0.0401907 ,
        0.0312376 , 0.02975559, 0.04164052, 0.03905511, 0.04262638]),
 'score_time': array([0.00598502, 0.00598431, 0.00598383, 0.00403237, 0.00348234,
        0.00400424, 0.00518513, 0.00599504, 0.00755978, 0.00600719]),
 'test_score': array([2.98695936e+04, 3.48894235e+04, 2.55698443e+04, 3.06872489e+04,
        3.67283216e+13, 4.36646863e+04, 4.15494153e+04, 3.97058499e+04,
        3.11623869e+04, 3.72696521e+04])}

In [34]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring=rmse_cv, n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

In [35]:
resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [36]:
resultados["reg_lineal_sin_seleccion"] = evaluar_modelo(LinearRegression(), ames_X, ames_y)
resultados["svr_sin_seleccion"] = evaluar_modelo(SVR(), ames_X, ames_y)
resultados["rf_sin_seleccion"] = evaluar_modelo(RandomForestRegressor(), ames_X, ames_y)

In [37]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.045761,0.006079,3.672832e+12,27932.676563,1.000000,1.000000,1.161274e+08,2.379629
svr_sin_seleccion,2.373622,0.316734,8.103164e+04,82149.044830,51.869804,52.102658,2.562054e+00,6.998408
rf_sin_seleccion,6.457177,0.026128,3.162761e+04,11738.248038,141.106063,4.298114,1.000000e+00,1.000000


Hay 3 tipos generales de estrategias de selección de variables:

# Métodos de filtrado

Los métodos de filtrado usan métodos estadísticos para seleccionar las variables que proporcionan la mayor cantidad de información. Estos métodos se aplican de forma previa a entrenar el modelo (preprocesado), y **son completamente independientes de la elección del estimador**. Generalmente funcionan definiendo una función de evaluación $S(xk_i, y_k)$, evaluando cada variable independiente para cada observación respecto a la variable objetivo de dicha observación, y eligiendo aquellas `K` variables que mejor funcionan.



Scikit-learn tiene las siguientes funciones de evaluación:

- Para regresión: `f_regression, mutual_info_regression`
- Para clasificación: `chi2, f_classif, mutual_info_classif`

f_regression y f_classif devuelven [estadísticos F](https://es.wikipedia.org/wiki/Estad%C3%ADstico_F) (F-values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresión, y un test ANOVA en el caso de clasificación.

`mutual_info_regression` y `mutual_info_classif` computan el coeficiente de información mutua (MIC) entre las variables independientes y la variable objetivo.

El coeficiente de información mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relación entre las mismas, y aumenta conforme más relación tienen.

El MIC se define como:

$$I(X;Y) = \sum_{x,y} P_{XY}(x,y) \log
{P_{XY}(x,y) \over P_X(x) P_Y(y)}$$

con $P_{XY}$ siendo la probabilidad conjunta de X e Y


El evaluador `chi2` calcula el estadístico chi cuadrado y lo convierte a estadísticos F, con el estadístico chi cuadrado definido como:
  $$\chi^2(x,y) = \sum {\frac{(x-y)^2}{y}}$$


In [38]:
from sklearn.feature_selection import SelectKBest, f_regression

In [39]:
selector_kbest10 = SelectKBest(f_regression, k=10)
ames_X_kbest10 = selector_kbest10.fit_transform(ames_X, ames_y)

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [40]:
ames_X_kbest10.shape

(2930, 10)

La funcion `get_support` nos devuelve un vector booleano (True/False), aquellos elementos con True son las columnas que se han seleccionado

In [41]:
columnas_seleccion_kbest10 = ames_X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

Index(['area_sotano_total', 'ano_construccion_garaje', 'area_garage',
       'tipo_andamios_PConc', 'calidad_cocinas', 'calidad_chimeneas',
       'acabado_garaje', 'calidad_material_exterior', 'altura_sotano',
       'calidad_calefaccion'],
      dtype='object')

El parámetro `scores_` del selector nos devuelve los resultados de la función de evaluación

In [42]:
selector_kbest10.scores_[:10]

array([1.68826400e+00, 6.57607119e+02, 3.87789909e+02, 8.83179807e-01,
       1.39328976e+01, 1.17035753e+03, 1.01901994e+02, 7.33188791e+00,
       2.54218342e+03, 7.41832196e+02])

In [43]:
puntuaciones_seleccion_kbest10 = zip(ames_X.columns, selector_kbest10.scores_,
                                     selector_kbest10.get_support())
evaluacion_kbest10 = sorted(
                filter(lambda c: c[2], puntuaciones_seleccion_kbest10),
                key=lambda c: c[1], reverse=True
)

In [44]:
list(evaluacion_kbest10)

[('calidad_material_exterior', 2781.4056293393337, True),
 ('ano_construccion_garaje', 2542.1834185458356, True),
 ('calidad_cocinas', 2423.00029861259, True),
 ('altura_sotano', 1372.464274306123, True),
 ('area_sotano_total', 1170.3575348657419, True),
 ('acabado_garaje', 1163.3451168655808, True),
 ('calidad_chimeneas', 1124.6033740020127, True),
 ('tipo_andamios_PConc', 1090.6972231859318, True),
 ('area_garage', 798.6223230530395, True),
 ('calidad_calefaccion', 744.8194902972515, True)]

Esto nos permite ver cual es la puntuación que le da el evaluador `f_regression` a cada variable independiente

In [45]:
resultados["reg_lineal_kbest_10"] = evaluar_modelo(LinearRegression(), ames_X_kbest10, ames_y)
resultados["rf_kbest_10"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest10, ames_y)
resultados["svr_kbest_10"] = evaluar_modelo(SVR(), ames_X_kbest10, ames_y)

In [46]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.045761,0.006079,3.672832e+12,27932.676563,11.294284,2.625124,1.161274e+08,2.379629
svr_sin_seleccion,2.373622,0.316734,8.103164e+04,82149.044830,585.832318,136.775914,2.562054e+00,6.998408
rf_sin_seleccion,6.457177,0.026128,3.162761e+04,11738.248038,1593.692023,11.283080,1.000000e+00,1.000000
reg_lineal_kbest_10,0.004052,0.002316,4.087897e+04,41030.327579,1.000000,1.000000,1.292509e+00,3.495439
rf_kbest_10,1.303797,0.021704,3.646584e+04,13544.306691,321.789368,9.372416,1.152975e+00,1.153861
svr_kbest_10,0.687798,0.387065,8.086586e+04,82001.120562,169.755121,167.147249,2.556812e+00,6.985806


Vemos que al reducir las dimensiones a solo 10, los modelos lineales y RF funcionan bastante peor, aunque vemos que por otra parte las diferencias entre la evaluación de entrenamiento y test se han reducido en todos los casos. Esto es así por que al tener menos variables, hay mucha menos flexibilidad para sobreajustar (es decir, "memorizar" los datos de entrenamiento). SVM sin embargo funciona más o menos igual (de mal), pero los tiempos de entrenamiento se reducen considerablemente (de 3.9 segundos a 0.7).

Probamos ahora para 50 variables

In [47]:
selector_kbest50 = SelectKBest(f_regression, k=50)
ames_X_kbest50 = selector_kbest50.fit_transform(ames_X, ames_y)

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [48]:
resultados["reg_lineal_kbest_50"] = evaluar_modelo(LinearRegression(), ames_X_kbest50, ames_y)
resultados["rf_kbest_50"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest50, ames_y)
resultados["svr_kbest_50"] = evaluar_modelo(SVR(), ames_X_kbest50, ames_y)

In [49]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.045761,0.006079,3.672832e+12,27932.676563,11.294284,2.625124,1.161274e+08,2.379629
svr_sin_seleccion,2.373622,0.316734,8.103164e+04,82149.044830,585.832318,136.775914,2.562054e+00,6.998408
rf_sin_seleccion,6.457177,0.026128,3.162761e+04,11738.248038,1593.692023,11.283080,1.000000e+00,1.000000
reg_lineal_kbest_10,0.004052,0.002316,4.087897e+04,41030.327579,1.000000,1.000000,1.292509e+00,3.495439
rf_kbest_10,1.303797,0.021704,3.646584e+04,13544.306691,321.789368,9.372416,1.152975e+00,1.153861
svr_kbest_10,0.687798,0.387065,8.086586e+04,82001.120562,169.755121,167.147249,2.556812e+00,6.985806
reg_lineal_kbest_50,0.016819,0.004171,3.539288e+04,34199.152363,4.151011,1.801303,1.119050e+00,2.913480
rf_kbest_50,3.195402,0.022698,3.182523e+04,11745.617342,788.655222,9.801592,1.006248e+00,1.000628
svr_kbest_50,0.831234,0.188924,8.099006e+04,82109.367927,205.156325,81.583539,2.560739e+00,6.995028


Vemos que para 50 variables, los tres estimadores funcionan de forma similar al modelo entrenado con las 209 variables iniciales, sin embargo los tiempos de entrenamiento de los mismos se reducen considerablemente.

De forma similar a `SelectKBest`, donde le decimos al transformador el número de variables que queremos mantener, podemos usar [`SelectPercentile`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile) donde especificamos el porcentaje de variables sobre el total.

Por ejemplo, si solo quisieramos conservar el 10% de las variables (sin importar que tengamos 200 o 20000), podemos hacer lo siguiente:


In [50]:
from sklearn.feature_selection import SelectPercentile

selector_pct10 = SelectPercentile(f_regression, percentile=10)
ames_X_pct10 = selector_pct10.fit_transform(ames_X, ames_y)

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [51]:
ames_X_pct10.shape

(2930, 19)

El transformador nos selecciona 21 variables (el 10 por ciento de 209 redondeado)

# Métodos envolventes (wrapper methods)

Los métodos envolventes *(wrapper methods)* funcionan de forma similar a los métodos de ranking. Sin embargo, en lugar de usar una función estadística independiente del modelo para evaluar las variables, estos métodos usan la función de evaluación o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador). Ésto significa que los métodos de filtrado se pueden aplicar independientemente de la elección del modelo, ya que consideran los modelos como una caja negra que produce evaluaciones, aunque claro, diferentes modelos producirán diferentes selecciones de variables.

`Scikit-learn` implementa un metodo envolvente llamado **Recursive Feature Elimination [(RFE)](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)**, o Eliminación Recursiva de Variables. RFE funciona seleccionando todas las variables, entrenando el modelo, usando los coeficientes `coef_` o la importancia de las variables `feature_importances_` en función del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el número de variables deseado.

In [52]:
from sklearn.feature_selection import RFE
estimador_selector = RandomForestRegressor()
selector_rfe10_rf = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_rf = selector_rfe10_rf.fit_transform(ames_X, ames_y)

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

In [53]:
ames_X_rfe10_rf.shape

(2930, 10)

In [54]:
evaluacion_rfe10_rf = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_rf.ranking_,
            selector_rfe10_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [55]:
evaluacion_rfe10_rf

[('perimetro_conectado_calle', 1, True),
 ('area_revestimiento', 1, True),
 ('area_sotano_total', 1, True),
 ('ano_construccion_garaje', 1, True),
 ('n_coches_garaje', 1, True),
 ('area_garage', 1, True),
 ('calidad_cocinas', 1, True),
 ('calidad_chimeneas', 1, True),
 ('calidad_material_exterior', 1, True),
 ('altura_sotano', 1, True)]

In [56]:
resultados["reg_lineal_rfe10_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_rf, ames_y)
resultados["rf_rfe10_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_rf, ames_y)
resultados["svr_rfe10_rf"] = evaluar_modelo(SVR(), ames_X_rfe10_rf, ames_y)

Si usamos otro estimador para evaluar veremos que las variables elegidas pueden ser completamente distintas. Los estimadores que se pueden usar tienen que implementar el metodo `coef_` o el metodo `feature_importance` (es decir, tienen que tener una manera de ordenar variables en función de su importancia). Por ejemplo, no podemos usar SVMs.

In [57]:
estimador_selector = LinearRegression()
selector_rfe10_lineal = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_lineal = selector_rfe10_lineal.fit_transform(ames_X, ames_y)

In [58]:
evaluacion_rfe10_lineal = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_lineal.ranking_,
            selector_rfe10_lineal.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)
evaluacion_rfe10_lineal

[('perimetro_conectado_calle', 1, True),
 ('area_sotano_habitable1', 1, True),
 ('area_sotano_habitable2', 1, True),
 ('area_sotano_inhabitable', 1, True),
 ('area_sotano_total', 1, True),
 ('n_aseos_sotanos', 1, True),
 ('n_medios_aseos_sotanos', 1, True),
 ('ano_construccion_garaje', 1, True),
 ('n_coches_garaje', 1, True),
 ('material_tejado_WdShngl', 1, True)]

In [59]:
set(evaluacion_rfe10_rf).intersection(set(evaluacion_rfe10_lineal))

{('ano_construccion_garaje', 1, True),
 ('area_sotano_total', 1, True),
 ('n_coches_garaje', 1, True),
 ('perimetro_conectado_calle', 1, True)}

In [60]:
resultados["reg_lineal_rfe10_lineal"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_lineal, ames_y)
resultados["rf_rfe10_lineal"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_lineal, ames_y)
resultados["svr_rfe10_lineal"] = evaluar_modelo(SVR(), ames_X_rfe10_lineal, ames_y)

In [61]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.045761,0.006079,3.672832e+12,27932.676563,11.294284,2.708243,1.161274e+08,2.379629
svr_sin_seleccion,2.373622,0.316734,8.103164e+04,82149.044830,585.832318,141.106684,2.562054e+00,6.998408
rf_sin_seleccion,6.457177,0.026128,3.162761e+04,11738.248038,1593.692023,11.640339,1.000000e+00,1.000000
reg_lineal_kbest_10,0.004052,0.002316,4.087897e+04,41030.327579,1.000000,1.031663,1.292509e+00,3.495439
rf_kbest_10,1.303797,0.021704,3.646584e+04,13544.306691,321.789368,9.669177,1.152975e+00,1.153861
svr_kbest_10,0.687798,0.387065,8.086586e+04,82001.120562,169.755121,172.439674,2.556812e+00,6.985806
reg_lineal_kbest_50,0.016819,0.004171,3.539288e+04,34199.152363,4.151011,1.858339,1.119050e+00,2.913480
rf_kbest_50,3.195402,0.022698,3.182523e+04,11745.617342,788.655222,10.111942,1.006248e+00,1.000628
svr_kbest_50,0.831234,0.188924,8.099006e+04,82109.367927,205.156325,84.166739,2.560739e+00,6.995028
reg_lineal_rfe10_rf,0.006960,0.002245,3.998163e+04,39993.569950,1.717902,1.000000,1.264137e+00,3.407116


Vemos que el estimador RF entrenado con solo 10 variables seleccionadas por `RFE` basado en RF es el modelo que mejor funciona, y tarda 8 veces menos en entrenar.

scikit-learn también tiene la implementación [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) que implementa el mismo algoritmo pero hace validación cruzada y entrena en múltiples particiones del dataset para calcular automáticamente el conjunto de variables que funciona mejor (a costa de un mayor tiempo de procesamiento).

In [62]:
from sklearn.feature_selection import RFECV

estimador_selector = RandomForestRegressor()
selector_rfecv = RFECV(estimador_selector)
ames_X_rfecv = selector_rfecv.fit_transform(ames_X, ames_y)

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

...

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:241: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:266: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X[:, features], y)
C:\Users\LCM\Anaconda3\lib\site-packages\sklearn\feature_selection\_rfe.py:629: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(self.transform(X), y)


In [63]:
ames_X_rfecv.shape

(2930, 112)

En este caso `RFECV` ha elegido 188 variables como aquellas que proporcionan el mejor funcionamiento al estimador RF utilizado.

In [64]:
resultados["reg_lineal_rfecv_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfecv, ames_y)
resultados["rf_rfecv_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfecv, ames_y)
resultados["svr_rfecv_rf"] = evaluar_modelo(SVR(), ames_X_rfecv, ames_y)

In [65]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.045761,0.006079,3.672832e+12,27932.676563,11.294284,2.708243,1.161274e+08,2.379629
svr_sin_seleccion,2.373622,0.316734,8.103164e+04,82149.044830,585.832318,141.106684,2.562054e+00,6.998408
rf_sin_seleccion,6.457177,0.026128,3.162761e+04,11738.248038,1593.692023,11.640339,1.000000e+00,1.000000
reg_lineal_kbest_10,0.004052,0.002316,4.087897e+04,41030.327579,1.000000,1.031663,1.292509e+00,3.495439
rf_kbest_10,1.303797,0.021704,3.646584e+04,13544.306691,321.789368,9.669177,1.152975e+00,1.153861
svr_kbest_10,0.687798,0.387065,8.086586e+04,82001.120562,169.755121,172.439674,2.556812e+00,6.985806
reg_lineal_kbest_50,0.016819,0.004171,3.539288e+04,34199.152363,4.151011,1.858339,1.119050e+00,2.913480
rf_kbest_50,3.195402,0.022698,3.182523e+04,11745.617342,788.655222,10.111942,1.006248e+00,1.000628
svr_kbest_50,0.831234,0.188924,8.099006e+04,82109.367927,205.156325,84.166739,2.560739e+00,6.995028
reg_lineal_rfe10_rf,0.006960,0.002245,3.998163e+04,39993.569950,1.717902,1.000000,1.264137e+00,3.407116


Vemos que el método que funciona mejor es el de RFECV usando RF como estimador. Este método funciona mejor que  el segundo mejor modelo, un Random Forest entrenado con todo el dataset, pero se entrena en menos tiempo (0.81 segundos versus  0.99 segundos)

# Metodos embebidos (Embedded Methods)

Éstos métodos son metodos de selección de variables que están integrados dentro del proceso de entrenamiento de modelos, y son específicos para cada modelo.

El ejemplo clásico de método embebidos de selección de variables son los procesos de regularización en regresión lineal. Por ejemplo el método L1 tiende a convertir los coeficientes de las variables que no funcionan a 0, por lo tanto eliminandolas.